The individual mandatory tour frequency model **predicts the number of work and school tours taken by each person with a mandatory DAP**. The primary drivers of mandatory tour frequency are demographics, accessibility-based parameters such as drive time to work, and household automobile ownership.

In [1]:
import os
import toml
import pandas as pd
import validation_data_input
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data
land_use = pd.read_csv(config['p_survey_landuse'])

per_data = validation_data_input.get_persons_data(['distance_to_work','cdap_activity','is_worker','is_student'])
hh_data = validation_data_input.get_households_data(['auto_ownership'])
tour_data = validation_data_input.get_tours_data(['tour_type','tour_category'])

# keep persons with mandatory tours
per_data = per_data.loc[per_data['cdap_activity']=="M"].\
    merge(hh_data[['household_id','auto_ownership','source']], how='left', on=['household_id','source']) # get auto ownership from hh data

# Create distance to work bins: bins of 2 miles up to 46 miles
max_bin = 46
bin_size = 2
per_data['d_work_bin_46mi'] = pd.cut(per_data['distance_to_work'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

tour_data = tour_data.loc[tour_data['person_id'].isin(per_data['person_id'])].\
    merge(per_data, how='left', on=['person_id','household_id','source'])

# total number of persons by source
df_person_count = per_data.groupby('source')['hh_weight_2017_2019'].sum().reset_index()

## mandatory tours per person

In [3]:
df_plot = tour_data.loc[tour_data['tour_category']=="mandatory"].\
    groupby(['source','tour_category'])[['trip_weight_2017_2019']].sum().reset_index(). \
    merge(df_person_count, how='left', on='source')
df_plot['tour_cat_rate'] = df_plot['trip_weight_2017_2019']/df_plot['hh_weight_2017_2019']

fig = px.bar(df_plot, x="tour_category", y="tour_cat_rate", color="source",barmode="group", template="simple_white",
             # color_discrete_sequence=config['psrc_color'],
             title="mandatory tour rates <br>(persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=400, font=dict(size=11))
fig.show()

In [4]:
# total number of worker/student by source
df_worker_count = per_data.loc[per_data['is_worker']].groupby('source')['hh_weight_2017_2019'].sum().reset_index()
df_worker_count['tour_type'] = "work"
df_worker_count['person_type'] = "worker"

df_student_count = per_data.loc[per_data['is_student']].groupby('source')['hh_weight_2017_2019'].sum().reset_index()
df_student_count['tour_type'] = "school"
df_student_count['person_type'] = "student"

df_worker_student_count = pd.concat([df_worker_count,df_student_count])

In [5]:

df_plot = tour_data.loc[tour_data['tour_type'].isin(["work","school"])].\
    groupby(['source','tour_type'])[['trip_weight_2017_2019']].sum().reset_index(). \
    merge(df_worker_student_count, how='left', on=['source','tour_type'])
df_plot['tour_cat_rate'] = df_plot['trip_weight_2017_2019']/df_plot['hh_weight_2017_2019']

fig = px.bar(df_plot, x="tour_type", y="tour_cat_rate", color="source",barmode="group", template="simple_white",
             facet_col='person_type',
             title="school/work tour rates per student/worker")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=500, font=dict(size=11))
fig.update_xaxes(matches=None)
fig.show()

## number of mandatory tours per person by segment

In [6]:
df_1 = per_data.groupby(['source','auto_ownership'])['hh_weight_2017_2019'].sum().reset_index()
df_2 = tour_data.loc[tour_data['tour_category']=="mandatory"].groupby(['source','auto_ownership'])['trip_weight_2017_2019'].sum().reset_index()

# plot1
df_plot = df_1.merge(df_2, how='left', on=['source','auto_ownership'])
df_plot['tour_rate'] = df_plot['trip_weight_2017_2019']/df_plot['hh_weight_2017_2019']

fig = px.bar(df_plot, x="auto_ownership", y="tour_rate", color="source",barmode="group", template="simple_white",
             # color_discrete_sequence=config['psrc_color'],
             title="mandatory tour rates by auto ownership (persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11))
fig.update_xaxes(dtick=1)
fig.show()

In [7]:
df_1 = per_data.groupby(['source','d_work_bin_46mi'])['hh_weight_2017_2019'].sum().reset_index()
df_2 = tour_data.loc[tour_data['tour_category']=="mandatory"].groupby(['source','d_work_bin_46mi'])['trip_weight_2017_2019'].sum().reset_index()

# plot1
df_plot = df_1.merge(df_2, how='left', on=['source','d_work_bin_46mi'])
df_plot['tour_rate'] = df_plot['trip_weight_2017_2019']/df_plot['hh_weight_2017_2019']


fig = px.line(df_plot, x='d_work_bin_46mi', y="tour_rate", color="source", template="simple_white",
               title="mandatory tour rates by distance to work (persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11))
fig.show()